## Imports

In [1]:
import torch
import shutil
import random
import glob

import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image

from torch import optim
from torch import nn

from matplotlib.patches import Rectangle

import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder
from torchvision.datasets import ImageNet
from torchvision import transforms
from torchvision import models

import torchvision


import matplotlib.pyplot as plt

## Data sorting

In [2]:
######## CARS #########
f1  = open("class_string.txt", "r").read().split("\n")
f2  = open("fname_string.txt", "r").read().split("\n")
f3 = open("bbox.txt", "r").read().split("\n")
totbbox = []
for ind, line in enumerate(f3):
    f1[ind] = int(f1[ind])
    bbox = line.split("\t") # x1,y1,x2,y2
    bbox[0] = float(bbox[0])
    bbox[1] = float(bbox[1])
    bbox[2] = float(bbox[2])
    bbox[3] = float(bbox[3])
    totbbox.append(bbox)
fin_list = []
for i in range(len(f2)):
    fin_list.append([f2[i],f1[i],totbbox[i][0],totbbox[i][1],totbbox[i][2],totbbox[i][3]])
df = pd.DataFrame(fin_list, columns=['image_id','labels', 'x0','y0','x1','y1'])

In [3]:
######## License plates ##########
f4 = open("lp_bb.txt", "r").read().split("\n")
totbbox = []
for line in f4:
    if(line==""):
        break
    bbox = line.split("\t") # x1,y1,x2,y2
    bbox[0] = float(bbox[0])
    bbox[1] = float(bbox[1])
    bbox[2] = float(bbox[2])
    bbox[3] = float(bbox[3])
    totbbox.append(bbox)
fin_list = []
for i in range(len(f4)-1):
    fin_list.append(["Cars{}.png".format(i),1,totbbox[i][0],totbbox[i][1],totbbox[i][2],totbbox[i][3]])
df_LP = pd.DataFrame(fin_list[:300], columns=['image_id','labels', 'x0','y0','x1','y1'])
df_LP_eval = pd.DataFrame(fin_list[300:350], columns=['image_id','labels', 'x0','y0','x1','y1'])
df_LP_test = pd.DataFrame(fin_list[350:], columns=['image_id','labels', 'x0','y0','x1','y1'])

In [4]:
######## Full_set ##########
df_tot=df.append(df_LP)

In [5]:
####### TEST_SET ########
ft = open("test.txt", "r").read().split("\n")
totbbox = []
fin_list = []
for ind, line in enumerate(ft):
    line = line.split("\t")
    if(int(line[6]) == 1):
        line[5] = int(line[5])
        bbox = line[1:5] # x1,y1,x2,y2
        bbox[0] = float(bbox[0])
        bbox[1] = float(bbox[1])
        bbox[2] = float(bbox[2])
        bbox[3] = float(bbox[3])
        totbbox.append(bbox)
        fin_list.append([line[0],line[5],bbox[0],bbox[1],bbox[2],bbox[3]])
random.shuffle(fin_list)    
df_test = pd.DataFrame(fin_list[0:7000], columns=['image_id','labels', 'x0','y0','x1','y1'])
df_eval = pd.DataFrame(fin_list[7001:], columns=['image_id','labels', 'x0','y0','x1','y1'])
df_test_full = df_test.append(df_LP_test)
df_eval_full = df_eval.append(df_LP_eval)

## Data Handler


In [6]:
class CarDataset(Dataset):
    def __init__(self, df, image_dir, transforms= None):
        super().__init__()
        
        self.df = df
        self.image_ids = self.df['image_id']
        self.image_dir = image_dir
        self.labels = self.df['labels']
        
    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        records = self.df[self.df["image_id"]==image_id]
        
        img = Image.open(self.image_dir/image_id).convert("RGB")
        img = transforms.ToTensor()(img)
        
        boxes = records[["x0","y0", "x1", "y1"]].values
        boxes = torch.tensor(boxes)
        boxes = boxes.type(torch.FloatTensor)
        labels = records["labels"].values
        labels = torch.tensor(labels)
        labels = labels.type(torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        
        return img, target
                            
                                  
                                  
    def __len__(self):
        return self.image_ids.shape[0]
    
def collate_fn(batch):
    return tuple(zip(*batch))  
        
dataset = CarDataset(df, Path.cwd() / "cars_train") #"train_all"
dataset_LP = CarDataset(df_LP,Path.cwd() / "LPtrain/images")        
dataloaded = DataLoader(dataset, batch_size=1, collate_fn = collate_fn)  
dataloaded_LP = DataLoader(dataset_LP, batch_size=1, collate_fn = collate_fn)

dataset_LP_eval = CarDataset(df_LP_eval,Path.cwd() / "LPtrain/images")
dataset_LP_test = CarDataset(df_LP_test,Path.cwd() / "LPtrain/images")   
dataloaded_LP_eval = DataLoader(dataset_LP_eval, batch_size=1, collate_fn = collate_fn)
dataloaded_LP_test = DataLoader(dataset_LP_test, batch_size=1, collate_fn = collate_fn)

dataset_tot = CarDataset(df_tot, Path.cwd() / "tot_train")
dataloaded_tot = DataLoader(dataset_tot, batch_size=1, collate_fn = collate_fn, shuffle = True)


dataset_tot_eval = CarDataset(df_eval_full, Path.cwd() / "eval_test")
dataset_tot_test = CarDataset(df_test_full, Path.cwd() / "eval_test")
dataloaded_tot_eval = DataLoader(dataset_tot_eval, batch_size=1, collate_fn = collate_fn, shuffle = True)
dataloaded_tot_test = DataLoader(dataset_tot_test, batch_size=1, collate_fn = collate_fn, shuffle = True)

## Create model

In [ ]:
###### CAR ########
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = models.detection.fasterrcnn_resnet50_fpn(pretrained = True, pretrained_backbone = True)#, num_classes = 197)

classes = 197
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features,classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

load_old_model = False

if(load_old_model):
    model.load_state_dict(torch.load(Path.cwd() / "one_epoch"))



In [7]:
###### License plate ########
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model_LP = models.detection.fasterrcnn_resnet50_fpn(pretrained = True, pretrained_backbone = True)#, num_classes = 197)

classes = 2
in_features = model_LP.roi_heads.box_predictor.cls_score.in_features
model_LP.roi_heads.box_predictor = FastRCNNPredictor(in_features,classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_LP.to(device)

load_old_model_LP = False

if(load_old_model_LP):
    model_LP.load_state_dict(torch.load(Path.cwd() / ""))


In [ ]:
###### Tot set ########
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model_tot = models.detection.fasterrcnn_resnet50_fpn(pretrained = True, pretrained_backbone = True)#, num_classes = 197)

classes = 198
in_features = model_tot.roi_heads.box_predictor.cls_score.in_features
model_tot.roi_heads.box_predictor = FastRCNNPredictor(in_features,classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_tot.to(device)

load_old_model_tot = False

if(load_old_model_tot):
    model_tot.load_state_dict(torch.load(Path.cwd() / "Model_tot_epoch_19"))


## Training


In [8]:
def evaluate_model(model, df):
    model.eval()
    n=0
    with torch.no_grad():
        for b_x, b_y in df:
            b_x = [item.to(device) for item in b_x]
            b_y = [{key: values.to(device) for key, values in target.items()} for target in b_y]
            output=model(b_x)
            if(output[0]['labels'].size()[0] > 0):
                if(output[0]['labels'][0] == b_y[0]['labels'][0]):
                    n+=1
    return n/len(df)

In [ ]:
###### CAR ######
model.train()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 16
train_acc = []
evaluate_acc = []
train_avg_loss = []
for epoch in range(num_epochs):
    n=0
    train_loss = []
    for i,(b_x, b_y) in enumerate(dataloaded):
        model.train()
        b_x = [item.to(device) for item in b_x]
        b_y = [{key: values.to(device) for key, values in target.items()} for target in b_y]
        loss_dict = model(b_x,b_y)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        train_loss.append(losses.item())
        model.eval()
        with torch.no_grad():
            output = model(b_x)
            if(output[0]['labels'].size()[0] > 0):
                if(output[0]['labels'][0] == b_y[0]['labels'][0]):
                    n+=1
        if(i%100 == 0):
            print(losses)
    train_avg_loss.append(sum(train_loss)/len(train_loss))
    evaluate_acc.append(evaluate_model(model,dataloaded_tot_eval))
    train_acc.append(n/(8144))
    lr_scheduler.step()
    print("epoch: {}\t train_avg_loss: {}\t train_acc: {}\t evaluation_acc: {}\n".format(epoch,train_avg_loss[epoch],train_acc[epoch],evaluate_acc[epoch]))
    torch.save(model.state_dict(), "models/Model_epoch_{}".format(epoch))
with open('acc_data.txt', 'w') as f:
    for i in range(len(train_avg_loss)):
        f.write("{}\t{}\t{}\n".format(train_avg_loss[i],train_acc[i],evaluate_acc[i]))
    

In [9]:
###### License plate ########

#model_LP.train()

#params = [p for p in model_LP.parameters() if p.requires_grad]
#optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
#num_epochs = 1
#for epoch in range(num_epochs):
#    for i,(b_x, b_y) in enumerate(dataloaded_LP):
#        b_x = [item.to(device) for item in b_x]
#        b_y = [{key: values.to(device) for key, values in target.items()} for target in b_y]
#        loss_dict=model_LP(b_x,b_y)
#        losses = sum(loss for loss in loss_dict.values())
#        optimizer.zero_grad()
#        losses.backward()
#        optimizer.step()
#        if(i%10 == 0):
#            print(losses)
#    lr_scheduler.step()
#    print(epoch)

#save_model_LP = False
#if(save_model_LP):
#    torch.save(model_LP.state_dict(), "Model_1")
    
model_LP.train()
params = [p for p in model_LP.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 16
train_acc = []
evaluate_acc = []
train_avg_loss = []
for epoch in range(num_epochs):
    n=0
    train_loss = []
    for i,(b_x, b_y) in enumerate(dataloaded_LP):
        model_LP.train()
        b_x = [item.to(device) for item in b_x]
        b_y = [{key: values.to(device) for key, values in target.items()} for target in b_y]
        loss_dict = model_LP(b_x,b_y)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        train_loss.append(losses.item())
        model_LP.eval()
        with torch.no_grad():
            output = model_LP(b_x)
            if(output[0]['labels'].size()[0] > 0):
                if(output[0]['labels'][0] == b_y[0]['labels'][0]):
                    n+=1
        if(i%40 == 0):
            print(losses)
    train_avg_loss.append(sum(train_loss)/len(train_loss))
    evaluate_acc.append(evaluate_model(model_LP,dataloaded_LP_eval))
    train_acc.append(n/(300))
    lr_scheduler.step()
    print("epoch: {}\t train_avg_loss: {}\t train_acc: {}\t evaluation_acc: {}\n".format(epoch,train_avg_loss[epoch],train_acc[epoch],evaluate_acc[epoch]))
    torch.save(model_LP.state_dict(), "models/Model_epoch_LP_{}".format(epoch))
with open('acc_data_LP.txt', 'w') as f:
    for i in range(len(train_avg_loss)):
        f.write("{}\t{}\t{}\n".format(train_avg_loss[i],train_acc[i],evaluate_acc[i]))

tensor(0.7874, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0800, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0819, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0382, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0335, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0565, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0565, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1639, device='cuda:0', grad_fn=<AddBackward0>)
epoch: 0	 train_avg_loss: 0.09698943298310042	 train_acc: 0.98	 evaluation_acc: 1.0

tensor(0.0621, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0400, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0437, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0334, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0243, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0369, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0380, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1043, device='cuda:0', grad_fn=<AddBackward0>)
epoch: 1	 train_avg

tensor(0.0228, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0199, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0132, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0227, device='cuda:0', grad_fn=<AddBackward0>)
epoch: 15	 train_avg_loss: 0.024883516182502112	 train_acc: 1.0	 evaluation_acc: 0.94



In [ ]:
###### Tot set ######

model_tot.train()

params = [p for p in model_tot.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
num_epochs = 1
train_acc = []
evaluate_acc = []
train_avg_loss = []
for epoch in range(num_epochs):
    n=0
    train_loss = []
    for i,(b_x, b_y) in enumerate(dataloaded_tot):
        model_tot.train()
        b_x = [item.to(device) for item in b_x]
        b_y = [{key: values.to(device) for key, values in target.items()} for target in b_y]
        loss_dict = model_tot(b_x,b_y)
        model_tot.eval()
        with torch.no_grad():
            output = model_tot(b_x)
            if(output[0]['labels'].size()[0] > 0):
                if(output[0]['labels'][0] == b_y[0]['labels'][0]):
                    n+=1
        model_tot.train()
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        train_loss.append(losses.item())
        if(i%100 == 0):
            print(losses)
    train_avg_loss.append(sum(train_loss)/len(train_loss))
    evaluate_acc.append(evaluate_model(model_tot,dataloaded_tot_eval))
    train_acc.append(n/(8144+433))
    lr_scheduler.step()
    print("epoch: {}\t train_avg_loss: {}\t train_acc: {}\t evaluation_acc: {}\n".format(epoch,train_avg_loss[epoch],train_acc[epoch],evaluate_acc[epoch]))
    torch.save(model_tot.state_dict(), "models/Model_tot_epoch_{}".format(epoch))

with open('acc_data_tot.txt', 'w') as f:
    for i in range(len(train_avg_loss)):
        f.write("{}\t{}\t{}\n".format(train_avg_loss[i],train_acc[i],evaluate_acc[epoch]))
save_model_tot = False
if(save_model_LP):
    torch.save(model_tot.state_dict(), "Model_tot_1")

In [ ]:
evaluate_avg_loss.append(evaluate_model(model_tot,dataloaded_tot_eval))

## Evaluation

In [ ]:
class_names  = open("class_names.txt", "r").read().split("\t")

def eval_and_plot_image(model, df, indx,plot_image=False, plate_on_cars = False):
    model = model.eval()
    image, target = df[indx]
    images = torch.stack([image]+[image]).to(device)
    with torch.no_grad():
        out=model(images)
    if(plot_image):
        fig, ax = plt.subplots()
        ax.imshow(image.permute(1, 2, 0))
        boxes=[]
        box = out[0]['boxes'][0]
        width=(box[2]-box[0])
        height=(box[3]-box[1])
        
    keep = torchvision.ops.nms(out[0]['boxes'], out[0]['scores'], 0.2)
    license_indxes = []
    car_indxes = []
    keep_final = []
    license_belong = torch.ones([len(out[0]['boxes'])])
    for indx in keep:
        if out[0]['labels'][indx] == 197 and out[0]['scores'][indx] > 0.2:
            license_indxes.append(indx)
        elif out[0]['labels'][indx] != 197 and out[0]['scores'][indx] > 0.1:
            keep_going = True
            car_indxes.append(indx)        
            keep_final.append(indx)
    
    
    ids = 0
    if plate_on_cars and 197 in out[0]['labels'] and len(keep)>1:
        for license_indx in license_indxes:
            keep_license = False
            for car_indx in car_indxes:

                if out[0]['boxes'][car_indx][0] < out[0]['boxes'][license_indx][0] and out[0]['boxes'][car_indx][2] > out[0]['boxes'][license_indx][2] and out[0]['boxes'][car_indx][1] < out[0]['boxes'][license_indx][1] and out[0]['boxes'][car_indx][3] > out[0]['boxes'][license_indx][3]:
                    ids += 1
                    license_belong[license_indx] = ids
                    license_belong[car_indx] = ids
                    keep_license = True
                    

            if keep_license:
                keep_final.append(license_indx) 
                        
    
                    
        
            

    for indx in keep_final:
        s = "Car: {:.0f}, Label: {}, score: {score:.2f}".format(license_belong[indx],class_names[out[0]['labels'][indx]-1],score=out[0]['scores'][indx])
        if(plot_image):
            width=(out[0]['boxes'][indx][2]-out[0]['boxes'][indx][0])
            height=(out[0]['boxes'][indx][3]-out[0]['boxes'][indx][1])
            ax.add_patch(Rectangle((out[0]['boxes'][indx][0],out[0]['boxes'][indx][1]),width,height,fill = False, edgecolor = "green"))
            ax.text(out[0]['boxes'][indx][0],out[0]['boxes'][indx][1],s,color = "red", size = 'small', fontweight = 'bold', horizontalalignment='left')


eval_and_plot_image(model_tot, dataset_tot, 8144+71, True, True) #8144+71

In [ ]:
#eval_and_plot_image(model, dataset, 29,True)
indices=[]
for i in range(100):        
    lab = eval_and_plot_image(model_tot, dataset_tot, i)
    indices.append(lab)
with open('50proc_tot_small.txt', 'w') as f:
    for item in indices:
        f.write("{}\n".format(item))

In [ ]:
eval_and_plot_image(model_tot, dataset_tot, 4, True,True)

In [ ]:
import xml.dom.minidom
def get_BB_xml(path):
    document = xml.dom.minidom.parse(path)
    x1 = document.getElementsByTagName("xmin")[0]
    x1=x1.childNodes[0]
    x1=x1.nodeValue
    y1 = document.getElementsByTagName("ymin")[0]
    y1=y1.childNodes[0]
    y1=y1.nodeValue
    x2 = document.getElementsByTagName("xmax")[0]
    x2=x2.childNodes[0]
    x2=x2.nodeValue
    y2 = document.getElementsByTagName("ymax")[0]
    y2=y2.childNodes[0]
    y2=y2.nodeValue
    return torch.tensor([float(x1),float(y1),float(x2),float(y2)],dtype=torch.float32)
bbox=[]
for i in range(433):
    bbox.append(get_BB_xml("LPtrain/annotations/Cars{}.xml".format(i)))
with open('lp_bb.txt', 'w') as f:
    for item in bbox:
        f.write("{}\t{}\t{}\t{}\n".format(item[0],item[1],item[2],item[3]))



In [ ]:
##### SCORE TEST SET #####
def score_model(model, dataset, indx):
    model = model.eval()
    image, target = dataset[indx]
    images = torch.stack([image]+[image]).to(device)
    with torch.no_grad():
        out=model(images)
        keep = torchvision.ops.nms(out[0]['boxes'], out[0]['scores'], 0.1)
        for indx in keep:
            if(target['labels'][0].to(device) == out[0]['labels'][indx]):# or out[0]['labels'][indx] == 197:
                return 1
    return 0
score=0           
for i in range(100):   
    score+=score_model(model_tot,dataset_tot_test,i)
print(score/100)